In [2]:
%%capture

UsageError: %%capture is a cell magic, but the cell body is empty.


In [3]:
%%capture
!pip install gdown
!pip install transformers
!pip install sentencepiece # MarianTokenizer 불러올 때 필요
!pip install sacremoses # MarianMTModel 에서 불러올 때 warning 뜨는 것 방지
!pip install einops # 지리는 einops 쓰기

In [4]:
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd
from tqdm import tqdm
import math
from einops import rearrange

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


In [5]:
# Load the tokenizer & input embedding layer & last fc layer
tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ko-en')
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-ko-en') # MT: Machine Translation
# input_embedding = model.get_input_embeddings() # 일부 사이즈 큰 놈들은 데려와서 나머지만 학습시키려는 시도
# input_embedding.weight.requires_grad = False # freeze
# fc_out = model.get_output_embeddings()
# fc_out.weight.requires_grad = False

In [6]:
eos_idx = tokenizer.eos_token_id
pad_idx = tokenizer.pad_token_id
print("eos_idx = ", eos_idx)
print("pad_idx = ", pad_idx)

eos_idx =  0
pad_idx =  65000


# 하이퍼파라미터 조정

In [7]:
BATCH_SIZE = 64 # 논문에선 2.5만 token이 한 batch에 담기게 했다고 함.
LAMBDA = 0 # l2-Regularization를 위한 hyperparam. # 저장된 모델
EPOCH = 15 # 저장된 모델
# max_len = 512 # model.model.encoder.embed_positions 를 보면 512로 했음을 알 수 있다.
max_len = 100 # 너무 긴거 같아서 자름 (GPU 부담도 많이 덜어짐)
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx) # pad token 이 출력 나와야하는 시점의 loss는 무시 (즉, label이 <pad> 일 때는 무시) # 저장된 모델
# criterion = nn.CrossEntropyLoss(ignore_index = pad_idx, label_smoothing = 0.1) # 막생 해보니 성능 안나옴 <- 데이터가 많아야 할 듯

scheduler_name = 'Noam'
# scheduler_name = 'Cos'
#### Noam ####
# warmup_steps = 4000 # 이건 논문에서 제시한 값 (총 10만 step의 4%)
warmup_steps = 1500 # 데이터 수 * EPOCH / BS = 총 step 수 인것 고려 # 저장된 모델
LR_scale = 0.5 # Noam scheduler에 peak LR 값 조절을 위해 곱해질 녀석 # 저장된 모델
#### Cos ####
LR_init = 5e-4
T0 = 1500 # 첫 주기
T_mult = 2 # 배 만큼 주기가 길어짐 (1보다 큰 정수여야 함)
#############

new_model_train = False
hyuk_model_use = True # 여러분만의 모델 만들어서 사용하고 싶다면 False로
if hyuk_model_use:
    !gdown https://drive.google.com/uc?id=16YgP0smjPPVhW7gm2BdvTfbRays2umYs -O Transformer_small.pt
    !gdown https://drive.google.com/uc?id=1-4nc6N2d3DR_LBDfGOtTHBptFIXAiFbO -O Transformer_small_history.pt
    save_model_path = 'Transformer_small.pt'
    save_history_path = 'Transformer_small_history.pt'
else:
    save_model_path = '/content/drive/MyDrive/Colab Notebooks/results/Transformer_small2.pt'
    save_history_path = '/content/drive/MyDrive/Colab Notebooks/results/Transformer_small2_history.pt'

Downloading...
From (uriginal): https://drive.google.com/uc?id=16YgP0smjPPVhW7gm2BdvTfbRays2umYs
From (redirected): https://drive.google.com/uc?id=16YgP0smjPPVhW7gm2BdvTfbRays2umYs&confirm=t&uuid=fe9ac7ad-20ed-494a-a453-27d258b04a50
To: /home/minyoungxi/MINYOUNGXI/Paper/Attention/Transformer_small.pt
100%|████████████████████████████████████████| 448M/448M [00:26<00:00, 17.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-4nc6N2d3DR_LBDfGOtTHBptFIXAiFbO
To: /home/minyoungxi/MINYOUNGXI/Paper/Attention/Transformer_small_history.pt
100%|██████████████████████████████████████████| 775/775 [00:00<00:00, 3.15MB/s]


In [8]:
vocab_size = tokenizer.vocab_size
print(vocab_size)

# 논문에 나오는 base 모델 (train loss를 많이 줄이려면 많은 Epoch이 요구됨, 또, test 성능도 좋으려면 더 많은 데이터 요구)
# n_layers = 6
# d_model = 512
# d_ff = 2048
# n_heads = 8
# drop_p = 0.1

# 좀 사이즈 줄인 모델 (훈련된 input_embedding, fc_out 사용하면 사용 불가)
n_layers = 3
d_model = 256
d_ff = 512
n_heads = 8
drop_p = 0.1

65001


# 토크나이저 & 학습된 모델 써보기

In [9]:
# tokenizer 써보기 (_로 띄어쓰기를 나타낸다! 즉, _가 없으면 이어진 한 단어임을 나타냄 subword tokenizing)
# 토크나이저에 대해 참고 자료: https://ratsgo.github.io/nlpbook/docs/preprocess/bpe/
print(tokenizer.tokenize("Hi, I'm Hyuk. ...        a   a?"))
print(tokenizer.tokenize("a/b 1+2+3 2:1 a>b"))
print(tokenizer.tokenize("pretrained restart"))
print(tokenizer.tokenize("chatGPT"))
print(tokenizer.tokenize("The example is very good in our lecture")) # 띄어쓰기도 tokenize 할 때가 있다.
print(tokenizer.tokenize("한글은 어떻게 할까?"))
print(tokenizer.tokenize("확실히 띄어쓰기 기준으로 토크나이징을 하는 것 같진 않다."))
print(tokenizer.tokenize("조사는? 나는 너는 우리는 강의는 너와 나의 내가 너가 나무가 헿"))
print(tokenizer.tokenize("마음대로 추가 문장"))

['▁H', 'i', ',', '▁I', "'", 'm', '▁H', 'y', 'u', 'k', '.', '▁', '...', '▁a', '▁a', '?']
['▁a', '/', 'b', '▁1', '+2', '+3', '▁2:', '1', '▁a', '>', 'b']
['▁p', 're', 'tra', 'in', 'ed', '▁re', 'st', 'art']
['▁ch', 'at', 'G', 'P', 'T']
['▁The', '▁', 'ex', 'am', 'ple', '▁is', '▁', 'very', '▁good', '▁in', '▁', 'our', '▁', 'le', 'c', 'ture']
['▁한', '글', '은', '▁어떻게', '▁할까', '?']
['▁확실히', '▁띄', '어', '쓰기', '▁기준으로', '▁토', '크', '나이', '징', '을', '▁하는', '▁것', '▁같', '진', '▁않다', '.']
['▁조사', '는', '?', '▁나는', '▁너는', '▁우리는', '▁강의', '는', '▁너와', '▁나의', '▁내가', '▁너가', '▁나무가', '▁', '헿']
['▁마음대로', '▁추가', '▁문장']


In [10]:
print(tokenizer.get_vocab())
print(tokenizer.vocab_size)

print(tokenizer.encode('지능', add_special_tokens=False)) # string to index
print(tokenizer.encode('<pad>', add_special_tokens=False)) # <pad>는 65000
print(tokenizer.encode('</s>', add_special_tokens=False)) # <sos> or <eos>는 0
print(tokenizer.encode('He', add_special_tokens=False)) # add_special_tokens=False 는 <eos> 자동 붙여주는 것을 방지
print(tokenizer.encode('he', add_special_tokens=False)) # 대소문자 다른 단어로 인식
print(tokenizer.tokenize('문장을 넣으면 토크나이즈해서 숫자로 바꾼다'))
print(tokenizer.encode('문장을 넣으면 토크나이즈해서 숫자로 바꾼다', add_special_tokens=False))
print(tokenizer.decode([204]))
print(tokenizer.decode([206]))
print(tokenizer.decode([210]))
print(tokenizer.decode(list(range(15)) + [65000,65001,65002,65003]))

{'</s>': 0, '<unk>': 1, '.': 2, ',': 3, '▁the': 4, '▁to': 5, '▁of': 6, '?': 7, '▁and': 8, '▁': 9, 's': 10, '▁a': 11, "'": 12, '▁in': 13, '▁들': 14, '▁that': 15, '▁I': 16, '▁”': 17, '▁you': 18, '▁“': 19, '▁is': 20, '▁이': 21, '▁-': 22, '▁을': 23, '▁it': 24, '▁의': 25, '▁for': 26, '▁은': 27, '!': 28, '▁에': 29, '▁’': 30, '▁be': 31, '▁‘': 32, '▁with': 33, '▁not': 34, '▁was': 35, '▁have': 36, '▁we': 37, '▁are': 38, '▁on': 39, '▁가': 40, '▁는': 41, '▁this': 42, '▁를': 43, '▁그': 44, '▁—': 45, ':': 46, '’': 47, '이': 48, 't': 49, '▁as': 50, '을': 51, ')': 52, '의': 53, '▁한': 54, '는': 55, '▁they': 56, '▁The': 57, '▁(': 58, '▁he': 59, '▁his': 60, '▁will': 61, '가': 62, '▁God': 63, '에': 64, '▁from': 65, '...': 66, '▁can': 67, '▁by': 68, '▁your': 69, '▁And': 70, '▁what': 71, '▁do': 72, '▁all': 73, '▁에서': 74, '▁me': 75, '▁으로': 76, '▁their': 77, '▁or': 78, '를': 79, '▁one': 80, '▁who': 81, '▁them': 82, '▁at': 83, '▁my': 84, '고': 85, '▁[': 86, '▁You': 87, '▁과': 88, '▁로': 89, '▁Jehovah': 90, '▁It': 91, '은': 92, '▁

In [11]:
# 사전 학습된 모델로 번역해보기 (생각보다 성능 좋네)
input_text = "헐! 대박 쩐다!"

input_tokens = tokenizer.encode(input_text, return_tensors="pt")
translated_tokens = model.generate(input_tokens, max_new_tokens=max_len)
translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

print("입력:", input_text)
print("AI의 번역:", translated_text)

입력: 헐! 대박 쩐다!
AI의 번역: Oh, my God! That's awesome!


# DS, DL 생성 테스트

In [14]:
# data 다운
# https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=126 에서 받을 수 있어요
!gdown https://drive.google.com/uc?id=14lAjaR2dRp5p5kEsm5GnwNM9KH-VgoOq -O 대화체.xlsx

Downloading...
From: https://drive.google.com/uc?id=14lAjaR2dRp5p5kEsm5GnwNM9KH-VgoOq
To: /home/minyoungxi/MINYOUNGXI/Paper/Attention/대화체.xlsx
100%|██████████████████████████████████████| 9.57M/9.57M [00:00<00:00, 22.1MB/s]


In [15]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data.loc[idx, '원문'], self.data.loc[idx, '번역문']

data = pd.read_excel('대화체.xlsx')
custom_DS = CustomDataset(data)

train_DS, val_DS, test_DS, _ = torch.utils.data.random_split(custom_DS, [95000, 2000, 1000, len(custom_DS)-95000-2000-1000])
# 논문에서는 450만개 영,독 문장 pair 사용

train_DL = torch.utils.data.DataLoader(train_DS, batch_size=BATCH_SIZE, shuffle=True)
val_DL = torch.utils.data.DataLoader(val_DS, batch_size=BATCH_SIZE, shuffle=True)
test_DL = torch.utils.data.DataLoader(test_DS, batch_size=BATCH_SIZE, shuffle=True)

print(len(train_DS))
print(len(val_DS))
print(len(test_DS))

95000
2000
1000


In [16]:
# test_DS 테스트
i = 5
idx = test_DS.indices[i]
print(idx) # 엑셀 파일에서 idx번째 문장에 들어있음을 확인할 수 있다
src_text, trg_text = custom_DS.__getitem__(idx)
print(src_text)
print(trg_text)

29160
여기는 제 자리인 거 같은데요?
I think this seat is mine.


In [17]:
# train_DL 테스트
src_texts, trg_texts = next(iter(train_DL)) # 데이터 한 국자 뜨기

print(src_texts)
print(trg_texts)
print(len(src_texts))
print(len(trg_texts))

# 여러 문장에 대해서는 tokenizer.encode() 가 아닌 그냥 tokenizer()
src = tokenizer(src_texts, padding=True, truncation=True, max_length = max_len, return_tensors='pt').input_ids # pt: pytorch tensor로 변환
# add_special_tokens = True (default)면 마지막 토큰에 <eos> 가 붙어 나옴
# truncation = True: max_len 보다 길면 끊고 <eos> 집어넣어버림
# src에 <eos>가 있는 게 반드시 좋은 건지는 알 수 없지만 그냥 붙여봤어요..
trg_texts = ['</s> ' + s for s in trg_texts]
trg = tokenizer(trg_texts, padding=True, truncation=True, max_length = max_len, return_tensors='pt').input_ids

print(src[:2])
print(trg[:2])
print(src.shape)
print(trg.shape)
print(trg[:,-1]) # 가장 마지막 단어를 보니 어떤 문장은 <eos> 로 끝이 났고 나머지는 <pad> 로 끝이 났다는 걸 볼 수 있음
print(tokenizer.decode(trg[trg[:,-1]==eos_idx,:][0])) # 가장 긴 문장 중 첫 번째 문장 관찰
print(trg[5,:-1]) # 디코더 입력
print(trg[5,1:]) # 디코더 출력
# 그런데 [:,:-1] 로 주면 패딩된 문장은 eos도 넣는 셈 아닌가? 맞다! 하지만 괜찮다. 어차피 출력으로 pad token이 기다리고 있으니.. (loss에서 ignore됨)

('네, 원래 매우 설탕이 많이 들어가는 음식입니다.', '그냥 작업에 몰두한 사람들, 여유를 즐기는 사람들 다 멋있어.', '그럼 그 오른쪽에 있는 신발은 재고가 있나요?', '컴퓨터를 올려놓을 거치대 같은 것이 필요합니다.', '항공권을 발권할 때 여권만 주시면 됩니다.', '네, 서울타워가 보이는 객실로 예약 확정되었습니다.', '분실신고 되었고, 사용 중지 처리가 되었습니다.', '예약 가능하신데 대신 침대를 하나 추가하셔야 하고 추가 금액은 1만 원입니다.', '어떤 게 이상한 지 말씀해주시겠습니까?', '어제 크레인 업체에 잘 다녀오셨습니까?', '그렇다면 제가 전화로 예약을 도와 드리겠습니다.', '너 요즘 연차 많이 썼잖아, 있어도 몇 개 없을 것 같은데.', '침착하시고 우선 높은 곳으로 올라가시기 바랍니다.', '아니요, 버스를 어디에서 타든 요금은 똑같습니다.', '늦은 시간에 도착하긴 하겠지만 오늘 내에는 받으실 수 있을 거예요.', '나는 우리 아버지 때문에 그 팀을 좋아하게 됐지.', '그렇다면 참가하겠으니, 주소와 시간 알려주세요.', '심지어 극 중에서도 20살 차이라니, 정말 현실성 없어.', '오늘이 두 번째 접종이시죠?', '네, 콧속에 뿌리는 스프레이가 있습니다.', '메일은 확인을 잘 하지 않으니 지금 말해주세요.', '저와 우리 아이들이 묵을만한 방이 혹시 있으신가요?', '드라마나 영화 보면 젊은 사람들 귀농 많이 하잖아.', '많이 아플 텐데요, 어쩌면 등산화가 익숙하지 않아서 그랬는지도 몰라요.', '이 작품은 이번 주말까지만 전시합니다.', '흡연 구역이 따로 있습니까?', '겉에 흠집이 있는 세탁기가 있긴 한데, 한번 보여 드릴게요.', '제가 알기로는 4월 15일부터 18일까지입니다.', '많이 늦어지는 않을 거예요, 연락 오는 대로 바로 알려 드리겠습니다.', '오늘은 줄넘기 500개를 해서 그런지 뭔가 벌써 건강해진 기분인데.', '확인 완료되셨고요, 할인 쿠폰을 발급해드릴 테니 이번 달 내에 써주세요.',

In [18]:
# 내가 쓸 train data 에 대해서 기존 model이 잘 번역하는지 확인
i = 5
idx = train_DS.indices[i]
src_text, trg_text = custom_DS.__getitem__(idx)
print(f"입력: {src_text}")
print(f"정답: {trg_text}")

src = tokenizer.encode(src_text, return_tensors='pt', add_special_tokens = True)
# add_special_tokens = False 해보면 뭔가 이상하게 번역함 (학습 때 source에도 <eos>를 넣었단 증거?)
translated_tokens = model.generate(src, max_new_tokens=max_len)
translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=False)

print(f"AI의 번역: {translated_text}") # 디코더 첫 입력으로 <pad> 토큰을 넣었음. (<pad>를 <sos>로 사용)

입력: 제가 직접 버스 타는 곳까지 안내해드릴게요.
정답: I'll guide you to the place where you can ride the bus.
AI의 번역: <pad> I'm going to show you where I'm going to take you to the bus.</s>
